<h1><center>GRID SEARCH И RANDOM SEARCH НА ПРАКТИКЕ</center></h1>

Импортируем библиотеки и установим параметры:

In [ ]:
import pandas as pd
import numpy as np
import os
import psycopg
import mlflow
from mlflow.tracking import MlflowClient
from catboost import CatBoostClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import ( OneHotEncoder, SplineTransformer, QuantileTransformer, StandardScaler,
                                    RobustScaler, PolynomialFeatures, KBinsDiscretizer )
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autofeat import AutoFeatRegressor, AutoFeatClassifier

import catboost as cb
from catboost import CatBoostClassifier
from category_encoders import CatBoostEncoder

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

TABLE_NAME = "clean_users_churn" # таблица с данными
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = 'model_grid_search' # ваш код здесь
RUN_NAME = 'model_grid_search' # ваш код здесь
REGISTRY_MODEL_NAME = 'model_grid_search' # ваш код здесь

experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)\
    if not mlflow.get_experiment_by_name(EXPERIMENT_NAME)\
    else mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

Загрузим таблицу:

In [ ]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {"host": 'rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net', #os.getenv("DB_DESTINATION_HOST"),
                        "port": '6432', #os.getenv("DB_DESTINATION_PORT"),
                        "dbname": 'playground_mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_NAME"),
                        "user": 'mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_USER"),
                        "password": '0c567edd8ad8472e87d5c85cc4d664e4' } #os.getenv("DB_DESTINATION_PASSWORD")}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
df['target'] = (df['end_date'].notna()).astype(int)
df.head(2) 

Подготовим данные для обучения:

In [ ]:
# Выделим признаки в три отдельные таблицы для дальнейшей работы:
features = df.drop(['customer_id','target'],axis=1)
num_features = features.select_dtypes(include=['float', 'int'])
date_features = features.select_dtypes(include='datetime64[ns]')
cat_features = features.select_dtypes(include='object')

# Посчитаем колво уникальных значений для катег. переменных и создадим создадим два датасета:
unique_values_per_col = cat_features.nunique().value_counts()
binary_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()==2] ]
other_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()!=2] ]

# Бинарные подразделяем на два - "да/нет" и другие бинарные:
yes_no_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()==True] ]
other_binary_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()!=True] ]

# Дубликаты
is_duplicated_id = df.duplicated(subset=['customer_id'], keep=False)

# Пропуски
cols_with_nans = df.isnull().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
for col in cols_with_nans:
    if df[col].dtype in [float, int]:
        fill_value = df[col].mean()
    elif df[col].dtype == 'object':
        fill_value = df[col].mode().iloc[0]
    df[col] = df[col].fillna(fill_value)

# Выбросы
num_cols = df.select_dtypes(['float']).columns
threshold = 1.5
potential_outliers = pd.DataFrame()
for col in num_cols:
	Q1 = df[col].quantile(0.25)
	Q3 = df[col].quantile(0.75)
	IQR = Q3 - Q1
	margin = threshold * IQR
	lower = Q1 - margin
	upper = Q3 + margin
	potential_outliers[col] = ~df[col].between(lower, upper)
outliers = potential_outliers.any(axis=1)

df.drop(columns=['id', 'customer_id', 'begin_date', 'end_date'], inplace=True)
df[-3:]

Первично обучим модель:

In [ ]:
features = ["monthly_charges", "total_charges", "senior_citizen"]
target = "target"

split_column = "monthly_charges" # ваш код здесь
stratify_column = target # ваш код здесь
test_size = 0.2 # ваш код здесь

df = df.sort_values(by=[split_column])

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target],\
                                   test_size=test_size, shuffle=False) # ваш код здесь

print(f"Размер выборки для обучения: {X_train.shape}")
print(f"Размер выборки для теста: {X_test.shape}")

In [ ]:

loss_function = ["Logloss"]
task_type = 'CPU'
random_seed = 0
iterations = 300
verbose = False

param_grid = {
    'loss_function': ['Logloss'],
    'task_type': [task_type],
    'random_seed': [random_seed],
    'iterations': [iterations],
    'verbose': [verbose]
}

model = CatBoostClassifier()  # Инициализация без передачи параметров
cv = GridSearchCV(model, param_grid=param_grid, cv=2, n_jobs=-1) # ваш код здесь

clf = cv.fit(X_train, y_train) # ваш код здесь

Проведём выбор гиперпараметров:

In [ ]:

os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net' #"https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE3Nlz8iDILW5VTYM1ihQB' #os.getenv("S3_ACCESS_KEY")
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44' #os.getenv("S3_SECRET_KEY")
os.environ['MLFLOW_ARTIFACT_URI'] = 'http://s3-student-mle-20250529-05fed48463'

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

cv_results = pd.DataFrame(clf.cv_results_) # ваш код здесь

best_params = clf.best_params_ # ваш код здесь

model_best = CatBoostClassifier(**best_params) #clf.best_params_ # ваш код здесь (объявите вашу модель с подобранными best_params)

model_best.fit(X_train, y_train)

prediction = model_best.predict(X_test)
probas = model_best.predict_proba(X_test)[:, 1]

# расчёт метрик качества
metrics = {}

_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel() # ошибки первого и второго рода
auc = roc_auc_score(y_test, probas) # площадь под ROC-кривой
precision = precision_score(y_test, prediction) # точность
recall = recall_score(y_test, prediction) # полнота
f1 = f1_score(y_test, prediction) # F1-мера
logloss = log_loss(y_test, prediction) # LogLoss

# сохранение метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

# дополнительные метрики из результатов кросс-валидации
metrics["mean_fit_time"] = cv_results['mean_fit_time'].mean() # среднее время обучения
metrics["std_fit_time"] = cv_results['std_fit_time'].mean() # стандартное отклонение времени обучения
metrics["mean_test_score"] = cv_results['mean_test_score'].mean() # средний результат на тесте
metrics["std_test_score"] = cv_results['std_test_score'].mean() # стандартное отклонение результата на тесте
metrics["best_score"] = clf.best_score_ # лучший результат кросс-валидации


Залогируем результаты:

In [ ]:

# настройки для логирования в MLFlow
pip_requirements= 'requirements.txt' # файл с зависимостями
signature = mlflow.models.infer_signature(X_test, prediction) # сигнатура модели
input_example = X_test[:10] # пример входных данных

# Проверьте, существует ли эксперимент, и создайте его, если нет
experiment_name = "model_grid_search"
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

model = CatBoostClassifier()
model.fit(X_train, y_train) 

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # ваш код здесь
    run_id = run.info.run_id
    mlflow.log_params(best_params)
    mlflow.log_metrics(metrics)
    cv_info = mlflow.sklearn.log_model(cv, artifact_path='cv')
    # mlflow.sklearn.log_model(model, REGISTRY_MODEL_NAME)
    mlflow.catboost.log_model(signature=signature,cb_model=model,\
    input_example=input_example, artifact_path='models',\
    registered_model_name=REGISTRY_MODEL_NAME,pip_requirements=pip_requirements)

**Задание 2**

Проделайте аналогичное упражнение, но в этот раз используйте алгоритм случайного поиска. Название run внутри эксперимента должно быть таким — model_random_search. Алгоритм отбора признаков залогируйте в директорию в объектном хранилище с таким же названием, как и в предыдущем задании —artifact_path="cv" . В переменную input_example положите первые 10 объектов из тестовой выборки.

In [ ]:
# TABLE_NAME = 'users_churn' # ваш код здесь

# TRACKING_SERVER_HOST = '127.0.0.1' # ваш код здесь
# TRACKING_SERVER_PORT = 5000 # ваш код здесь

# EXPERIMENT_NAME = 'model_grid_search' # ваш код здесь
# RUN_NAME = 'model_random_search'
# REGISTRY_MODEL_NAME = 'model_grid_search' # ваш код здесь

features = ["monthly_charges", "total_charges", "senior_citizen"]
target = "target"

split_column = "monthly_charges" # ваш код здесь
stratify_column = target # ваш код здесь
test_size = 0.2 # ваш код здесь

df = df.sort_values(by=[split_column])

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target],\
                                   test_size=test_size, shuffle=False) # ваш код здесь

print(f"Размер выборки для обучения: {X_train.shape}")
print(f"Размер выборки для теста: {X_test.shape}")

loss_function = "Logloss"
task_type = 'CPU'
random_seed = 0
iterations = 300
verbose = False

params = { 'loss_function': loss_function,
           'task_type': task_type,
           'random_seed': random_seed,
           'iterations': iterations,
           'verbose': verbose }

model = CatBoostClassifier(**params) #, loss_function=loss_function, loss_function=loss_function,
#iterations=iterations, verbose=verbose, random_seed=random_seed, task_type=task_type) # ваш код здесь

param_distributions = { 'loss_function': ['Logloss'], 'iterations': [10, 50, 100], 'depth': [3, 4, 5] }

cv = RandomizedSearchCV(model, param_distributions=param_distributions, n_iter=20, cv=2, n_jobs=-1) # ваш код здесь

clf = cv.fit(X_train, y_train) # ваш код здесь

# os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
# os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("S3_ACCESS_KEY")
# os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("S3_SECRET_KEY")


mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

cv_results = pd.DataFrame(clf.cv_results_) # ваш код здесь

best_params = clf.best_params_ # ваш код здесь

model = CatBoostClassifier(**best_params) #, loss_function=loss_function, loss_function=loss_function,
#iterations=iterations, verbose=verbose, random_seed=random_seed, task_type=task_type) # ваш код здесь (объявите вашу модель с подобранными best_params)

model.fit(X_train, y_train)

prediction = model.predict(X_test)
probas = model.predict_proba(X_test)[:, 1]

# расчёт метрик качества
metrics = {}

_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel() # ошибки первого и второго рода
auc = roc_auc_score(y_test, probas) # площадь под ROC-кривой
precision = precision_score(y_test, prediction) # точность
recall = recall_score(y_test, prediction) # полнота
f1 = f1_score(y_test, prediction) # F1-мера
logloss = log_loss(y_test, prediction) # LogLoss

# сохранение метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

# дополнительные метрики из результатов кросс-валидации
metrics["mean_fit_time"] = cv_results['mean_fit_time'].mean() # среднее время обучения
metrics["std_fit_time"] = cv_results['std_fit_time'].mean() # стандартное отклонение времени обучения
metrics["mean_test_score"] = cv_results['mean_test_score'].mean() # средний результат на тесте
metrics["std_test_score"] = cv_results['std_test_score'].mean() # стандартное отклонение результата на тесте
metrics["best_score"] = clf.best_score_ # лучший результат кросс-валидации

# настройки для логирования в MLFlow
pip_requirements= 'requirements.txt' # файл с зависимостями
signature = mlflow.models.infer_signature(X_test, prediction) # сигнатура модели
input_example = X_test[:10] # пример входных данных

model = CatBoostClassifier()
model.fit(X_train, y_train) 

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id # ваш код здесь

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # ваш код здесь
    run_id = run.info.run_id
    mlflow.log_params(best_params)
    mlflow.log_metrics(metrics)
    cv_info = mlflow.sklearn.log_model(cv, artifact_path='cv')
    mlflow.catboost.log_model(model,signature=signature,input_example=input_example,\
    artifact_path='models',registered_model_name=REGISTRY_MODEL_NAME,pip_requirements=pip_requirements)